# 🤖 Importar bibliotecas necessárias
Este notebook importa as mesmas bibliotecas utilizadas no treinamento e está preparado para o processo de previsão (forecasting).

In [2]:
# 🤖 Importar bibliotecas
import pandas as pd
import holidays

# 🤖 Carregar arquivo de inferência em DataFrame
A seguir, o arquivo 'ventas_2025_inferencia.csv' será carregado em um DataFrame chamado inferencia_df.

In [3]:
# 🤖 Carregar arquivo ventas_2025_inferencia em inferencia_df
inferencia_path = '../data/raw/inferencia/ventas_2025_inferencia.csv'
inferencia_df = pd.read_csv(inferencia_path)

# 🤖 Mostrar as primeiras linhas do DataFrame inferencia_df
inferencia_df.head()

,fecha,producto_id,nombre,categoria,subcategoria,precio_base,es_estrella,unidades_vendidas,precio_venta,ingresos,Amazon,Decathlon,Deporvillage
0,2025-10-25,PROD_001,Nike Air Zoom Pegasus 40,Running,Zapatillas Running,115,True,26.0,113.13,2941.38,89.51,113.43,104.78
1,2025-10-25,PROD_002,Adidas Ultraboost 23,Running,Zapatillas Running,135,True,27.0,141.89,3831.03,128.73,112.91,122.88
2,2025-10-25,PROD_003,Asics Gel Nimbus 25,Running,Zapatillas Running,85,False,5.0,85.79,428.95,84.28,74.51,85.57
3,2025-10-25,PROD_004,New Balance Fresh Foam X 1080v12,Running,Zapatillas Running,75,False,3.0,76.19,228.57,75.54,70.32,71.13
4,2025-10-25,PROD_005,Nike Dri-FIT Miler,Running,Ropa Running,35,False,3.0,35.48,106.44,33.84,31.32,34.41


In [4]:
# 🤖 Engenharia de variáveis para inferencia_df replicando a lógica do treinamento
# 1. Converter data para datetime
inferencia_df['fecha'] = pd.to_datetime(inferencia_df['fecha'])

# 2. Variáveis temporais e de calendário
inferencia_df['año'] = inferencia_df['fecha'].dt.year
inferencia_df['mes'] = inferencia_df['fecha'].dt.month
inferencia_df['dia_mes'] = inferencia_df['fecha'].dt.day
inferencia_df['dia_semana'] = inferencia_df['fecha'].dt.dayofweek
inferencia_df['nombre_dia_semana'] = inferencia_df['fecha'].dt.day_name()
inferencia_df['nombre_mes'] = inferencia_df['fecha'].dt.month_name()

# Feriados Brasil
br_holidays = holidays.Brazil(years=inferencia_df['año'].unique())
inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(br_holidays)

# Black Friday e Cyber Monday
from pandas import Timestamp

def es_black_friday(fecha):
    if fecha.month == 11:
        ultima_sexta = max([d for d in pd.date_range(start=f'{fecha.year}-11-01', end=f'{fecha.year}-11-30') if d.weekday() == 4])
        return fecha == ultima_sexta
    return False
inferencia_df['es_black_friday'] = inferencia_df['fecha'].apply(es_black_friday)

def es_cyber_monday(fecha):
    if fecha.month == 11 or fecha.month == 12:
        ultima_sexta = max([d for d in pd.date_range(start=f'{fecha.year}-11-01', end=f'{fecha.year}-11-30') if d.weekday() == 4])
        primeira_segunda = ultima_sexta + pd.Timedelta(days=(7 - ultima_sexta.weekday()))
        return fecha == primeira_segunda
    return False
inferencia_df['es_cyber_monday'] = inferencia_df['fecha'].apply(es_cyber_monday)

inferencia_df['semana_año'] = inferencia_df['fecha'].dt.isocalendar().week
inferencia_df['trimestre'] = inferencia_df['fecha'].dt.quarter
inferencia_df['es_fin_semana'] = inferencia_df['dia_semana'].isin([5,6])

# 3. Variáveis de desconto e preços
inferencia_df['descuento_porcentaje'] = ((inferencia_df['precio_venta'] - inferencia_df['precio_base']) / inferencia_df['precio_base']) * 100
compet_cols = [col for col in ['Amazon', 'Decathlon', 'Deporvillage'] if col in inferencia_df.columns]
if compet_cols:
    inferencia_df['precio_competencia'] = inferencia_df[compet_cols].mean(axis=1)
    inferencia_df['ratio_precio'] = inferencia_df['precio_venta'] / inferencia_df['precio_competencia']
    inferencia_df = inferencia_df.drop(columns=compet_cols)
else:
    print('As colunas de concorrentes já foram removidas ou não existem.')

# 4. Lags e médias móveis (sem dropna)
lags = range(1, 8)
def criar_lags_media(df):
    df = df.sort_values(['producto_id', 'fecha'])
    for lag in lags:
        df[f'unidades_vendidas_lag{lag}'] = df.groupby(['producto_id', 'año'])['unidades_vendidas'].shift(lag)
    df['unidades_vendidas_mm7'] = df.groupby(['producto_id', 'año'])['unidades_vendidas'].transform(lambda x: x.rolling(7).mean())
    return df
inferencia_df = criar_lags_media(inferencia_df)

# 5. One-hot encoding de variáveis categóricas
inferencia_df['nombre_h'] = inferencia_df['nombre']
inferencia_df['categoria_h'] = inferencia_df['categoria']
inferencia_df['subcategoria_h'] = inferencia_df['subcategoria']
categoricas = ['nombre_h', 'categoria_h', 'subcategoria_h']
inferencia_df = pd.get_dummies(inferencia_df, columns=categoricas, drop_first=False)

# 6. Filtrar apenas registros de novembro
inferencia_df = inferencia_df[inferencia_df['mes'] == 11].copy()

# 8. Salvar resultado
inferencia_df.to_csv('../data/processed/inferencia_df_transformado.csv', index=False)
print('🤖 DataFrame de inferência transformado e salvo em ../data/processed/inferencia_df_transformado.csv')
inferencia_df.shape

C:\Users\gemlopez\AppData\Local\Temp\ipykernel_17580\3722584318.py:15: FutureWarning: The behavior of 'isin' with dtype=datetime64[ns] and castable values (e.g. strings) is deprecated. In a future version, these will not be considered matching by isin. Explicitly cast to the appropriate dtype before calling isin instead.
  inferencia_df['es_festivo'] = inferencia_df['fecha'].isin(br_holidays)


🤖 DataFrame de inferência transformado e salvo em ../data/processed/inferencia_df_transformado.csv


(720, 77)